# Importação de bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

# Instância de conexão com o banco de dados

In [2]:
usuario = "root"
senha = "root"
url = "localhost"
porta = "3306"
banco = "ENEM_OLTP"

# Criar uma conexão com o banco de dados MySQL
engine = create_engine(f"mysql+pymysql://{usuario}:{senha}@{url}:{porta}/{banco}")

# Leitura do arquivo CSV com os microdados do ENEM 2023

Limita a quantidade de linhas a serem lidas para a quantidade na variável `MAXIMO`

In [3]:
MAXIMO = 1_000_000

df = pd.read_csv("../datasets/MICRODADOS_ENEM_2023.csv", sep=";", encoding="ISO-8859-1", nrows=MAXIMO)

display(df.head())

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,2023,14,M,2,1,1,1,17,1,...,C,C,B,B,A,B,B,A,A,B
1,210059527735,2023,12,M,2,1,0,1,16,1,...,B,A,B,B,A,A,C,A,D,B
2,210061103945,2023,6,F,1,1,1,1,0,1,...,B,A,A,B,A,A,A,A,A,B
3,210060214087,2023,2,F,1,3,1,2,0,2,...,A,A,A,B,A,A,D,A,A,B
4,210059980948,2023,3,F,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,A,A


# Dropa as colunas que não serão utilizadas

In [4]:
df.drop(
    columns=[
        # Dados da escola
        "CO_MUNICIPIO_ESC",
        "NO_MUNICIPIO_ESC",
        "CO_UF_ESC",
        "SG_UF_ESC",
        "TP_DEPENDENCIA_ADM_ESC",
        "TP_LOCALIZACAO_ESC",
        "TP_SIT_FUNC_ESC",
        # Dados do local de aplicação da prova
        "CO_MUNICIPIO_PROVA",
        "NO_MUNICIPIO_PROVA",
        "CO_UF_PROVA",
        "SG_UF_PROVA",
        # Dados da prova objetiva
        "CO_PROVA_CN",
        "CO_PROVA_CH",
        "CO_PROVA_LC",
        "CO_PROVA_MT",
        "TX_RESPOSTAS_CN",
        "TX_RESPOSTAS_CH",
        "TX_RESPOSTAS_LC",
        "TX_RESPOSTAS_MT",
        "TX_GABARITO_CN",
        "TX_GABARITO_CH",
        "TX_GABARITO_LC",
        "TX_GABARITO_MT",
        # Dados da redação
        "NU_NOTA_COMP1",
        "NU_NOTA_COMP2",
        "NU_NOTA_COMP3",
        "NU_NOTA_COMP4",
        "NU_NOTA_COMP5",
    ],
    inplace=True,
)

df.replace(np.nan, None, inplace=True)

display(df.head())

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,2023,14,M,2,1,1,1,17,1,...,C,C,B,B,A,B,B,A,A,B
1,210059527735,2023,12,M,2,1,0,1,16,1,...,B,A,B,B,A,A,C,A,D,B
2,210061103945,2023,6,F,1,1,1,1,0,1,...,B,A,A,B,A,A,A,A,A,B
3,210060214087,2023,2,F,1,3,1,2,0,2,...,A,A,A,B,A,A,D,A,A,B
4,210059980948,2023,3,F,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,A,A


# Criação da base com as informações do participante

In [5]:
df_dados_participante = df[
    [
        "NU_INSCRICAO",
        "NU_ANO",
        "TP_FAIXA_ETARIA",
        "TP_SEXO",
        "TP_ESTADO_CIVIL",
        "TP_COR_RACA",
        "TP_NACIONALIDADE",
        "TP_ST_CONCLUSAO",
        "TP_ANO_CONCLUIU",
        "TP_ESCOLA",
        "TP_ENSINO",
        "IN_TREINEIRO",
    ]
]

display(df_dados_participante.head())

with engine.connect() as connection:
    df_dados_participante.to_sql(
        "PARTICIPANTES", con=engine, if_exists="replace", index=False
    )
    connection.execute(
        text(
            """
            ALTER TABLE PARTICIPANTES
            ADD PRIMARY KEY (NU_INSCRICAO)
            """
        )
    )

    connection.execute(
        text(
            """
            ALTER TABLE PARTICIPANTES
            MODIFY COLUMN TP_SEXO CHAR(1)
            """
        )
    )

    connection.execute(
        text(
            """
            ALTER TABLE PARTICIPANTES
            MODIFY COLUMN TP_ENSINO BIGINT
            """
        )
    )

    connection.execute(
        text(
            """
            ALTER TABLE PARTICIPANTES
            ENGINE = InnoDB
            """
        )
    )

    print("Colunas modificadas com sucesso!")

    for coluna in df_dados_participante.columns[2:]:
        connection.execute(
            text(
                f"""
                ALTER TABLE PARTICIPANTES
                ADD CONSTRAINT FK_PARTICIPANTES_{coluna}
                FOREIGN KEY ({coluna}) REFERENCES {coluna[3:]}({coluna}) ON DELETE CASCADE
                """
            )
        )
    print("Chave estrangeira adicionada com sucesso!")

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO
0,210059085136,2023,14,M,2,1,1,1,17,1,None,0
1,210059527735,2023,12,M,2,1,0,1,16,1,None,0
2,210061103945,2023,6,F,1,1,1,1,0,1,None,0
3,210060214087,2023,2,F,1,3,1,2,0,2,1.0,0
4,210059980948,2023,3,F,1,3,1,2,0,2,1.0,0


Colunas modificadas com sucesso!
Chave estrangeira adicionada com sucesso!


# Dataframe com as informações da prova objetiva do participante

In [6]:
df_dados_prova_objetiva = df[
    [
        "NU_INSCRICAO",
        "TP_PRESENCA_CN",
        "TP_PRESENCA_CH",
        "TP_PRESENCA_LC",
        "TP_PRESENCA_MT",
        "NU_NOTA_CN",
        "NU_NOTA_CH",
        "NU_NOTA_LC",
        "NU_NOTA_MT",
        "TP_LINGUA",
    ]
]

display(df_dados_prova_objetiva.head())

,NU_INSCRICAO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA
0,210059085136,0,0,0,0,None,None,None,None,0
1,210059527735,0,0,0,0,None,None,None,None,0
2,210061103945,1,1,1,1,502.0,498.9,475.6,363.2,1
3,210060214087,1,1,1,1,459.0,508.5,507.2,466.7,0
4,210059980948,1,1,1,1,402.5,379.2,446.9,338.3,0


# Dataframe com as informações da redação do participante

In [7]:
df_dados_redacao = df[
    [
        "NU_INSCRICAO",
        "TP_STATUS_REDACAO",
        "NU_NOTA_REDACAO",
    ]
]

display(df_dados_redacao.head())

,NU_INSCRICAO,TP_STATUS_REDACAO,NU_NOTA_REDACAO
0,210059085136,None,None
1,210059527735,None,None
2,210061103945,1.0,700.0
3,210060214087,1.0,880.0
4,210059980948,1.0,560.0


# Criação da base com as informações da notas do participante

In [8]:
df_dados_notas = pd.merge(
    df_dados_prova_objetiva,
    df_dados_redacao,
    on="NU_INSCRICAO",
)

display(df_dados_notas.head())


colunas_notas = [
    coluna
    for coluna in df_dados_notas.columns
    if coluna.startswith("NU_") and not coluna.startswith("NU_INSCRICAO")
]

with engine.connect() as connection:
    # connection.execute(text(f"DROP TABLE IF EXISTS NOTAS"))
    df_dados_notas.to_sql("NOTAS", con=engine, if_exists="replace", index=False)
    for coluna in colunas_notas:
        connection.execute(
            text(f"ALTER TABLE NOTAS MODIFY COLUMN {coluna} DECIMAL(5, 1)")
        )
    connection.execute(
        text(f"ALTER TABLE NOTAS MODIFY COLUMN TP_STATUS_REDACAO BIGINT")
    )
    connection.execute(
        text(
            """
            ALTER TABLE NOTAS
            ENGINE = InnoDB
            """
        )
    )
    print("Colunas alteradas com sucesso!")
    connection.execute(
        text(
            """
            ALTER TABLE NOTAS
            ADD CONSTRAINT FK_NOTAS_NU_INSCRICAO
            FOREIGN KEY (NU_INSCRICAO) REFERENCES PARTICIPANTES(NU_INSCRICAO) ON DELETE CASCADE
            """
        )
    )
    for coluna in list(
        set(df_dados_notas.columns) - set(colunas_notas + ["NU_INSCRICAO"])
    ):
        connection.execute(
            text(
                f"""
                ALTER TABLE NOTAS
                ADD CONSTRAINT FK_NOTAS_{coluna}
                FOREIGN KEY ({coluna}) REFERENCES {coluna[3:]}({coluna}) ON DELETE CASCADE
                """
            )
        )
    print("Chaves estrangeiras adicionadas com sucesso!")

,NU_INSCRICAO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_REDACAO
0,210059085136,0,0,0,0,None,None,None,None,0,None,None
1,210059527735,0,0,0,0,None,None,None,None,0,None,None
2,210061103945,1,1,1,1,502.0,498.9,475.6,363.2,1,1.0,700.0
3,210060214087,1,1,1,1,459.0,508.5,507.2,466.7,0,1.0,880.0
4,210059980948,1,1,1,1,402.5,379.2,446.9,338.3,0,1.0,560.0


Colunas alteradas com sucesso!
Chaves estrangeiras adicionadas com sucesso!


# Criação da base com as informações do questionário sócio-econômico do participante

In [9]:
df_dados_questionario_socioeconomico = df[
    [
        "NU_INSCRICAO",
    ]
    + [f"Q{str(i).zfill(3)}" for i in range(1, 26)]
]

display(df_dados_questionario_socioeconomico.head())


with engine.connect() as connection:
    # connection.execute(text(f"DROP TABLE IF EXISTS QUESTIONARIO"))
    df_dados_questionario_socioeconomico.to_sql(
        "QUESTIONARIO", con=engine, if_exists="replace", index=False
    )
    for coluna in [f"Q{str(i).zfill(3)}" for i in range(1, 26)]:
        try:
            connection.execute(
                text(
                    f"""
                    ALTER TABLE QUESTIONARIO
                    MODIFY COLUMN {coluna} CHAR(1)
                    """
                )
            )
        except Exception as e:
            print(f"Coluna {coluna} falhou")

        connection.execute(
            text(
                """
                ALTER TABLE QUESTIONARIO
                ENGINE = InnoDB
                """
            )
        )

        print(f"Coluna {coluna} alterada com sucesso!")

        connection.execute(
            text(
                f"""
                ALTER TABLE QUESTIONARIO
                ADD CONSTRAINT FK_QUESTIONARIO_{coluna}
                FOREIGN KEY ({coluna}) REFERENCES RESP_QUESTAO_{int(coluna[1:])}({coluna}) ON DELETE CASCADE
                """
            )
        )

    connection.execute(
        text(
            f"""
            ALTER TABLE QUESTIONARIO
            ADD CONSTRAINT FK_QUESTIONARIO_NU_INSCRICAO
            FOREIGN KEY (NU_INSCRICAO) REFERENCES PARTICIPANTES(NU_INSCRICAO) ON DELETE CASCADE
            """
        )
    )

    print("Chaves estrangeiras adicionadas com sucesso!")

,NU_INSCRICAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,A,F,E,D,5,F,C,C,D,...,C,C,B,B,A,B,B,A,A,B
1,210059527735,F,E,E,B,3,H,A,B,C,...,B,A,B,B,A,A,C,A,D,B
2,210061103945,H,E,C,F,5,C,A,B,D,...,B,A,A,B,A,A,A,A,A,B
3,210060214087,D,D,B,B,5,C,A,B,B,...,A,A,A,B,A,A,D,A,A,B
4,210059980948,B,B,A,A,4,B,A,B,A,...,A,A,A,B,A,A,B,A,A,A


Coluna Q001 alterada com sucesso!
Coluna Q002 alterada com sucesso!
Coluna Q003 alterada com sucesso!
Coluna Q004 alterada com sucesso!
Coluna Q005 falhou
Coluna Q005 alterada com sucesso!
Coluna Q006 alterada com sucesso!
Coluna Q007 alterada com sucesso!
Coluna Q008 alterada com sucesso!
Coluna Q009 alterada com sucesso!
Coluna Q010 alterada com sucesso!
Coluna Q011 alterada com sucesso!
Coluna Q012 alterada com sucesso!
Coluna Q013 alterada com sucesso!
Coluna Q014 alterada com sucesso!
Coluna Q015 alterada com sucesso!
Coluna Q016 alterada com sucesso!
Coluna Q017 alterada com sucesso!
Coluna Q018 alterada com sucesso!
Coluna Q019 alterada com sucesso!
Coluna Q020 alterada com sucesso!
Coluna Q021 alterada com sucesso!
Coluna Q022 alterada com sucesso!
Coluna Q023 alterada com sucesso!
Coluna Q024 alterada com sucesso!
Coluna Q025 alterada com sucesso!
Chaves estrangeiras adicionadas com sucesso!
